In [252]:
## get data

In [249]:
import requests
import pandas as pd

In [250]:
html = "https://en.wikipedia.org/wiki/List_of_areas_of_London"

In [251]:
df = pd.read_html(html)[1]
df.head()

,Location,London borough,Post town,Postcode district,Dial code,OS grid ref
0,Abbey Wood,"Bexley, Greenwich [7]",LONDON,SE2,020,TQ465785
1,Acton,"Ealing, Hammersmith and Fulham[8]",LONDON,"W3, W4",020,TQ205805
2,Addington,Croydon[8],CROYDON,CR0,020,TQ375645
3,Addiscombe,Croydon[8],CROYDON,CR0,020,TQ345665
4,Albany Park,Bexley,"BEXLEY, SIDCUP","DA5, DA14",020,TQ478728


In [5]:
df.columns

Index(['Location', 'London borough', 'Post town', 'Postcode district',
       'Dial code', 'OS grid ref'],
      dtype='object')

In [6]:
df.columns = ['Location','London Borough','Post town','Postcode district','Dial Code','OS grid ref']

In [7]:
df.drop(columns=["Dial Code","OS grid ref"],inplace=True)

In [8]:
df['London Borough'] =  df['London Borough'].apply(lambda x: x.replace('[','').replace(']','')) 
df['London Borough'] =  df['London Borough'].str.replace('\d+', '')
df['London Borough'] =  df['London Borough'].str.split(',').str[0]
df['Postcode district'] =  df['Postcode district'].str.split(',').str[0]
df['Postcode district'] =  df['Postcode district'].str.split('(').str[0]
df['Post town'] =  df['Post town'].str.split(',').str[0]

In [9]:
df.head()

,Location,London Borough,Post town,Postcode district
0,Abbey Wood,Bexley,LONDON,SE2
1,Acton,Ealing,LONDON,W3
2,Addington,Croydon,CROYDON,CR0
3,Addiscombe,Croydon,CROYDON,CR0
4,Albany Park,Bexley,BEXLEY,DA5


In [10]:
df.shape

(531, 4)

In [11]:
#df_Lon = df[df['Post town'].str.contains('LONDON')] ... gives less data points
df_Lon = df

In [12]:
df_Lon.reset_index(drop=True, inplace=True)

In [13]:
df_Lon.head()

,Location,London Borough,Post town,Postcode district
0,Abbey Wood,Bexley,LONDON,SE2
1,Acton,Ealing,LONDON,W3
2,Addington,Croydon,CROYDON,CR0
3,Addiscombe,Croydon,CROYDON,CR0
4,Albany Park,Bexley,BEXLEY,DA5


In [14]:
df_Lon.shape

(531, 4)

In [15]:
## get longitude / latidude

In [16]:
import numpy as np # library to handle data in a vectorized manner
import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

In [17]:
pip install geocoder

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |████████████████████████████████| 98 kB 9.8 MB/s  eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [18]:
import geocoder

In [19]:
def getLatLong(row):
    #print('post :{}'.format(row[:]))
    #print('neigh :{}'.format(row[1]))
    # initialize your variable to None
    lat_lng_coords = None
    search_query = '{}, London,UK'.format(row)
    # loop until you get the coordinates
    try:
        while(lat_lng_coords is None):
            #g = geocoder.here(search_query,app_id=app_id,app_code=app_code)
            g = geocoder.arcgis(search_query)
            lat_lng_coords = g.latlng
            #print('FIRST')
    except IndexError:
        latitude = 0.0
        longitude = 0.0
        print('BACKUP')
        return [latitude,longitude]

    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    print(latitude, longitude)
    return [latitude, longitude]

In [20]:
coords_list = df_Lon['Postcode district'].apply(getLatLong).tolist()

51.492450000000076 0.12127000000003818
51.51324000000005 -0.2674599999999714
51.38475500000004 -0.05149847299992416
51.38475500000004 -0.05149847299992416
51.50642000000005 -0.1272099999999341
51.50642000000005 -0.1272099999999341
51.51200000000006 -0.08057999999994081
51.51651000000004 -0.11967999999995982
51.50642000000005 -0.1272099999999341
51.41009000000008 -0.05682999999993399
51.523610000000076 -0.09876999999994496
51.27179309706539 0.07472437841542252
51.56393000000003 -0.12944999999996298
51.58370000000008 0.17476000000004888
51.64441500000004 -0.1791830389999518
51.616310000000055 -0.1383899999999585
51.44822000000005 -0.1483899999999494
51.499960000000044 -0.09567999999995891
51.523610000000076 -0.09876999999994496
51.53312000000005 0.08407653200004006
51.50642000000005 -0.1272099999999341
51.47052000000008 0.1467047760000355
51.47469000000007 -0.24162999999992962
51.45202003021353 0.172281349876519
51.615680000000054 -0.2451099999999542
51.64441500000004 -0.1791830389999518

In [21]:
coords_list

[[51.492450000000076, 0.12127000000003818],
 [51.51324000000005, -0.2674599999999714],
 [51.38475500000004, -0.05149847299992416],
 [51.38475500000004, -0.05149847299992416],
 [51.50642000000005, -0.1272099999999341],
 [51.50642000000005, -0.1272099999999341],
 [51.51200000000006, -0.08057999999994081],
 [51.51651000000004, -0.11967999999995982],
 [51.50642000000005, -0.1272099999999341],
 [51.41009000000008, -0.05682999999993399],
 [51.523610000000076, -0.09876999999994496],
 [51.27179309706539, 0.07472437841542252],
 [51.56393000000003, -0.12944999999996298],
 [51.58370000000008, 0.17476000000004888],
 [51.64441500000004, -0.1791830389999518],
 [51.616310000000055, -0.1383899999999585],
 [51.44822000000005, -0.1483899999999494],
 [51.499960000000044, -0.09567999999995891],
 [51.523610000000076, -0.09876999999994496],
 [51.53312000000005, 0.08407653200004006],
 [51.50642000000005, -0.1272099999999341],
 [51.47052000000008, 0.1467047760000355],
 [51.47469000000007, -0.24162999999992962

In [22]:
from pandas import DataFrame
latlong = DataFrame (coords_list, columns=['Latitude', 'Longitude'])
latlong

,Latitude,Longitude
0,51.492450,0.121270
1,51.513240,-0.267460
2,51.384755,-0.051498
3,51.384755,-0.051498
4,51.506420,-0.127210
...,...,...
526,51.482070,0.071430
527,51.506420,-0.127210
528,51.506450,-0.236910
529,51.506420,-0.127210


In [23]:
# data frame for London with Latitude and Longitude values is called df_LLL
df_LLL = df_Lon.join(latlong)
df_LLL

,Location,London Borough,Post town,Postcode district,Latitude,Longitude
0,Abbey Wood,Bexley,LONDON,SE2,51.492450,0.121270
1,Acton,Ealing,LONDON,W3,51.513240,-0.267460
2,Addington,Croydon,CROYDON,CR0,51.384755,-0.051498
3,Addiscombe,Croydon,CROYDON,CR0,51.384755,-0.051498
4,Albany Park,Bexley,BEXLEY,DA5,51.506420,-0.127210
...,...,...,...,...,...,...
526,Woolwich,Greenwich,LONDON,SE18,51.482070,0.071430
527,Worcester Park,Sutton,WORCESTER PARK,KT4,51.506420,-0.127210
528,Wormwood Scrubs,Hammersmith and Fulham,LONDON,W12,51.506450,-0.236910
529,Yeading,Hillingdon,HAYES,UB4,51.506420,-0.127210


In [24]:
# address of German/Austrian/Swiss embassies, companies and schools in London
    # German - 23 Belgrave Square, Belgravia, London SW1X 8PZ
    # Austria - 18 Belgrave Mews W, Belgravia, London SW1X 8HU
    # Switzerland - 16-18 Montagu Pl, Marylebone, London W1H 2BQ
    # German School | Petersham Road | Richmond, Surrey | TW10 7AH
    # BASF - 110 Bishopsgate, London EC2N 4AY
    # Munich RE - 10 Fenchurch Avenue London EC3M 5BN
    # E.ON - 129 Wilton Road, London, SW1V 1JZ
    # SAP - Bedfont Road, Feltham, Middlesex TW14 8HD 
    # Erste Group - 110 Bishopsgate, London EC2N 4AY
    # OMV - Westminster, London SW1E 5DH
    # VoestAlpine - Hammersmith, London W6 0QT
    # Glencore - Mayfair, London W1J 8HD
    # UBS - London EC2M 2QS

In [25]:
ger_spots = ['German Embassy', 'Belgravia', 'SW1X'],['Austrian Embassy','Belgravia', 'SW1X'],['Swiss Embassy', 'Marylebone', 'W1H'],['German School', 'Richmond', 'TW10'],['BASF', 'London', 'EC2N'], ['Munich RE', 'London', 'EC3M'], ['E.ON', 'London', 'SW1V'], ['SAP', 'Feltham', 'TW14'],['Erste Group', 'London', 'EC2N'], ['OMV', 'Westminster', 'SW1E'], ['VoestAlpine', 'Hammersmith', 'W6'], ['Glencore', 'Mayfair', 'W1J'], ['UBS', 'London', 'EC2M']

In [26]:
df_ger = DataFrame (ger_spots, columns=['Entity', 'Location', 'Postcode district'])
df_ger

,Entity,Location,Postcode district
0,German Embassy,Belgravia,SW1X
1,Austrian Embassy,Belgravia,SW1X
2,Swiss Embassy,Marylebone,W1H
3,German School,Richmond,TW10
4,BASF,London,EC2N
5,Munich RE,London,EC3M
6,E.ON,London,SW1V
7,SAP,Feltham,TW14
8,Erste Group,London,EC2N
9,OMV,Westminster,SW1E


In [27]:
gcoords_list = df_ger['Postcode district'].apply(getLatLong).tolist()

51.50064500000008 -0.1509561889999418
51.50064500000008 -0.1509561889999418
51.520290000000045 -0.16173992499994938
51.44075000000004 -0.2820922619999351
51.51530500000007 -0.08349458299994694
51.51055500000007 -0.08225808299994242
51.49421930400007 -0.14280999999994037
51.44844804245899 -0.44849085603639316
51.51530500000007 -0.08349458299994694
51.49840000000006 -0.1420261039999673
51.49617000000006 -0.22934999999995398
51.508380000000045 -0.13880574999996043
51.516950000000065 -0.08333996199996818


In [28]:
glatlong = DataFrame (gcoords_list, columns=['Latitude', 'Longitude'])
glatlong

,Latitude,Longitude
0,51.500645,-0.150956
1,51.500645,-0.150956
2,51.520290,-0.161740
3,51.440750,-0.282092
4,51.515305,-0.083495
5,51.510555,-0.082258
6,51.494219,-0.142810
7,51.448448,-0.448491
8,51.515305,-0.083495
9,51.498400,-0.142026


In [29]:
df_gLLL = df_ger.join(glatlong)
df_gLLL

,Entity,Location,Postcode district,Latitude,Longitude
0,German Embassy,Belgravia,SW1X,51.500645,-0.150956
1,Austrian Embassy,Belgravia,SW1X,51.500645,-0.150956
2,Swiss Embassy,Marylebone,W1H,51.520290,-0.161740
3,German School,Richmond,TW10,51.440750,-0.282092
4,BASF,London,EC2N,51.515305,-0.083495
5,Munich RE,London,EC3M,51.510555,-0.082258
6,E.ON,London,SW1V,51.494219,-0.142810
7,SAP,Feltham,TW14,51.448448,-0.448491
8,Erste Group,London,EC2N,51.515305,-0.083495
9,OMV,Westminster,SW1E,51.498400,-0.142026


In [30]:
## map

In [304]:
!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

Solving environment: \ 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/noarch::ibm-wsrt-py37main-keep==0.0.0=2020
  - defaults/noarch::ibm-wsrt-py37main-main==custom=2020
  - conda-forge/linux-64::pytorch==1.8.0=cpu_py37hafa7651_0
done

# All requested packages already installed.



In [32]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

In [33]:
address = 'London'

geolocator = Nominatim(user_agent="ldn_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of London are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of London are 51.5073219, -0.1276474.


In [359]:
map_london = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_LLL['Latitude'], df_LLL['Longitude'], df_LLL['London Borough'], df_LLL['Location']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=1,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_london) 
    
    
for lat, lng, borough, neighborhood in zip(df_gLLL['Latitude'], df_gLLL['Longitude'], df_gLLL['Entity'], df_gLLL['Location']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=2,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_london)     
    
        
map_london

In [333]:
## Foursquare

In [371]:
CLIENT_ID = 'KWLUKLGNJPLHEZYM0VXPJ4ZGLWNNFHILYOGP5YXXFRMAY23M' # your Foursquare ID
CLIENT_SECRET = '1TTSXQLN12N2O2Z5VMHGNXZUARDHZEKITIM1S2HIDA0MLNEE' # your Foursquare Secret
#ACCESS_TOKEN = 'LPJDFVWX2QP4Z3HYMG4PJR0XJ3G3MCRNOZJAHNESMSML4R4F' # your FourSquare Access Token
VERSION = '20180604' #... from Lab
#VERSION = '20200605' ... from Forum
#VERSION = '20210303'
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: KWLUKLGNJPLHEZYM0VXPJ4ZGLWNNFHILYOGP5YXXFRMAY23M
CLIENT_SECRET:1TTSXQLN12N2O2Z5VMHGNXZUARDHZEKITIM1S2HIDA0MLNEE


In [372]:
! pip install folium==0.5.0
import folium # plotting library

In [373]:
# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize
from pandas import json_normalize

import json # library to handle JSON files

In [389]:
address = 'London'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

51.5073219 -0.1276474


In [390]:
search_query = 'German'
radius = 25000
LIMIT = 100
print(search_query + ' .... OK!')

German .... OK!


In [391]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url


'https://api.foursquare.com/v2/venues/search?client_id=KWLUKLGNJPLHEZYM0VXPJ4ZGLWNNFHILYOGP5YXXFRMAY23M&client_secret=1TTSXQLN12N2O2Z5VMHGNXZUARDHZEKITIM1S2HIDA0MLNEE&ll=51.5073219,-0.1276474&v=20180604&query=German&radius=25000&limit=100'

In [392]:
import requests
! pip install folium==0.5.0

In [393]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '6069d35c17146414eeec0b14'},
 'response': {'venues': [{'id': '4dde2ecdb0fbc2c4eefc6f76',
    'name': 'German Historical Institute',
    'location': {'address': '17 Bloomsbury Square',
     'lat': 51.520094212882555,
     'lng': -0.12326506261178953,
     'labeledLatLngs': [{'label': 'display',
       'lat': 51.520094212882555,
       'lng': -0.12326506261178953}],
     'distance': 1453,
     'postalCode': 'WC1 B 4',
     'cc': 'GB',
     'city': 'Camden Town',
     'state': 'Greater London',
     'country': 'United Kingdom',
     'formattedAddress': ['17 Bloomsbury Square',
      'Camden Town',
      'Greater London',
      'WC1 B 4',
      'United Kingdom']},
    'categories': [{'id': '4bf58dd8d48988d190941735',
      'name': 'History Museum',
      'pluralName': 'History Museums',
      'shortName': 'History Museum',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/museum_history_',
       'suffix': '.png'},
     

In [394]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet,location.neighborhood,venuePage.id
0,4dde2ecdb0fbc2c4eefc6f76,German Historical Institute,"[{'id': '4bf58dd8d48988d190941735', 'name': 'H...",v-1617548125,False,17 Bloomsbury Square,51.520094,-0.123265,"[{'label': 'display', 'lat': 51.52009421288255...",1453,WC1 B 4,GB,Camden Town,Greater London,United Kingdom,"[17 Bloomsbury Square, Camden Town, Greater Lo...",NaN,NaN,NaN
1,5735bed2498e60bccee659ef,German Doner Kebab,"[{'id': '5283c7b4e4b094cb91ec88d8', 'name': 'D...",v-1617548125,False,11A Islington High St,51.532408,-0.106242,"[{'label': 'display', 'lat': 51.53240786313911...",3161,N1 9LQ,GB,London,Greater London,United Kingdom,"[11A Islington High St, London, Greater London...",NaN,NaN,NaN
2,4b03ff8bf964a520cf5022e3,German Embassy London,"[{'id': '4bf58dd8d48988d12c951735', 'name': 'E...",v-1617548125,False,23 Belgrave Sq,51.497723,-0.154989,"[{'label': 'display', 'lat': 51.49772312574038...",2175,SW1X 8PZ,GB,London,Greater London,United Kingdom,"[23 Belgrave Sq (Chesham Pl), London, Greater ...",Chesham Pl,NaN,NaN
3,4c194cec9990d13acd357c17,German Deli (Food Stall),"[{'id': '4bf58dd8d48988d10d941735', 'name': 'G...",v-1617548125,False,Borough Market,51.505739,-0.089977,"[{'label': 'display', 'lat': 51.50573875782404...",2616,SE1 1TL,GB,London,Greater London,United Kingdom,"[Borough Market, London, Greater London, SE1 1...",NaN,NaN,NaN
4,5ff0c0d66a11745dd104a6e1,Buy German Driver License Online,"[{'id': '52e81612bcbc57f1066b7a42', 'name': 'D...",v-1617548125,False,72 Guild Street,51.509143,-0.125259,"[{'label': 'display', 'lat': 51.50914311161548...",261,SE22 8NY,GB,London,Greater London,United Kingdom,"[72 Guild Street, London, Greater London, SE22...",NaN,NaN,NaN


In [395]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
#filtered_columns = ['name', 'categories', 'location.lat', 'location.lng']
dataframe_filtered = dataframe.loc[:, filtered_columns]

In [344]:
dataframe_filtered

,name,categories,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet,location.neighborhood,id
0,German Historical Institute,"[{'id': '4bf58dd8d48988d190941735', 'name': 'H...",17 Bloomsbury Square,51.520094,-0.123265,"[{'label': 'display', 'lat': 51.52009421288255...",1453,WC1 B 4,GB,Camden Town,Greater London,United Kingdom,"[17 Bloomsbury Square, Camden Town, Greater Lo...",NaN,NaN,4dde2ecdb0fbc2c4eefc6f76
1,German Doner Kebab,"[{'id': '5283c7b4e4b094cb91ec88d8', 'name': 'D...",11A Islington High St,51.532408,-0.106242,"[{'label': 'display', 'lat': 51.53240786313911...",3161,N1 9LQ,GB,London,Greater London,United Kingdom,"[11A Islington High St, London, Greater London...",NaN,NaN,5735bed2498e60bccee659ef
2,German Embassy London,"[{'id': '4bf58dd8d48988d12c951735', 'name': 'E...",23 Belgrave Sq,51.497723,-0.154989,"[{'label': 'display', 'lat': 51.49772312574038...",2175,SW1X 8PZ,GB,London,Greater London,United Kingdom,"[23 Belgrave Sq (Chesham Pl), London, Greater ...",Chesham Pl,NaN,4b03ff8bf964a520cf5022e3
3,German Deli (Food Stall),"[{'id': '4bf58dd8d48988d10d941735', 'name': 'G...",Borough Market,51.505739,-0.089977,"[{'label': 'display', 'lat': 51.50573875782404...",2616,SE1 1TL,GB,London,Greater London,United Kingdom,"[Borough Market, London, Greater London, SE1 1...",NaN,NaN,4c194cec9990d13acd357c17
4,Buy German Driver License Online,"[{'id': '52e81612bcbc57f1066b7a42', 'name': 'D...",72 Guild Street,51.509143,-0.125259,"[{'label': 'display', 'lat': 51.50914311161548...",261,SE22 8NY,GB,London,Greater London,United Kingdom,"[72 Guild Street, London, Greater London, SE22...",NaN,NaN,5ff0c0d66a11745dd104a6e1
5,German Gymnasium,"[{'id': '52e81612bcbc57f1066b79f9', 'name': 'M...",1 King's Boulevard,51.532392,-0.125267,"[{'label': 'display', 'lat': 51.53239240571996...",2795,N1C 4BU,GB,London,Greater London,United Kingdom,"[1 King's Boulevard, London, Greater London, N...",NaN,Kings cross,4b792875f964a5204fed2ee3
6,Jost German Design,"[{'id': '4bf58dd8d48988d1e1931735', 'name': 'A...",13 Princes Arcade London,51.510170,-0.134421,"[{'label': 'display', 'lat': 51.51016997850715...",566,SW1Y 6DS,GB,London,NaN,United Kingdom,"[13 Princes Arcade London, London, SW1Y 6DS, U...",NaN,NaN,4f04aa02e4b04fbec49fd62d
7,Hooray’s German Hot Dogs,"[{'id': '4bf58dd8d48988d16f941735', 'name': 'H...",Covent Garden,51.511919,-0.122090,"[{'label': 'display', 'lat': 51.511919, 'lng':...",640,WC2E 8RE,GB,London,Greater London,United Kingdom,"[Covent Garden, London, Greater London, WC2E 8...",NaN,NaN,5a44f2cb2b98442bd7fbf450
8,German Doner Kebab,"[{'id': '5283c7b4e4b094cb91ec88d8', 'name': 'D...",7 Fulham Broadway,51.480196,-0.196337,"[{'label': 'display', 'lat': 51.480196, 'lng':...",5637,SW6 1AA,GB,London,Greater London,United Kingdom,"[7 Fulham Broadway, London, Greater London, SW...",NaN,NaN,583375e97ff1e41bfbb90e09
9,German National Tourist Office,"[{'id': '4f4530164b9074f6e4fb00ff', 'name': 'T...",NaN,51.508731,-0.144363,"[{'label': 'display', 'lat': 51.508731, 'lng':...",1168,NaN,GB,NaN,NaN,United Kingdom,[United Kingdom],NaN,NaN,4bfce0eabf6576b09373acb8


In [396]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [382]:
dataframe_filtered

,name,categories,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet,location.neighborhood,id
0,German Historical Institute,"[{'id': '4bf58dd8d48988d190941735', 'name': 'H...",17 Bloomsbury Square,51.520094,-0.123265,"[{'label': 'display', 'lat': 51.52009421288255...",1453,WC1 B 4,GB,Camden Town,Greater London,United Kingdom,"[17 Bloomsbury Square, Camden Town, Greater Lo...",NaN,NaN,4dde2ecdb0fbc2c4eefc6f76
1,German Doner Kebab,"[{'id': '5283c7b4e4b094cb91ec88d8', 'name': 'D...",11A Islington High St,51.532408,-0.106242,"[{'label': 'display', 'lat': 51.53240786313911...",3161,N1 9LQ,GB,London,Greater London,United Kingdom,"[11A Islington High St, London, Greater London...",NaN,NaN,5735bed2498e60bccee659ef
2,German Embassy London,"[{'id': '4bf58dd8d48988d12c951735', 'name': 'E...",23 Belgrave Sq,51.497723,-0.154989,"[{'label': 'display', 'lat': 51.49772312574038...",2175,SW1X 8PZ,GB,London,Greater London,United Kingdom,"[23 Belgrave Sq (Chesham Pl), London, Greater ...",Chesham Pl,NaN,4b03ff8bf964a520cf5022e3
3,German Deli (Food Stall),"[{'id': '4bf58dd8d48988d10d941735', 'name': 'G...",Borough Market,51.505739,-0.089977,"[{'label': 'display', 'lat': 51.50573875782404...",2616,SE1 1TL,GB,London,Greater London,United Kingdom,"[Borough Market, London, Greater London, SE1 1...",NaN,NaN,4c194cec9990d13acd357c17
4,Buy German Driver License Online,"[{'id': '52e81612bcbc57f1066b7a42', 'name': 'D...",72 Guild Street,51.509143,-0.125259,"[{'label': 'display', 'lat': 51.50914311161548...",261,SE22 8NY,GB,London,Greater London,United Kingdom,"[72 Guild Street, London, Greater London, SE22...",NaN,NaN,5ff0c0d66a11745dd104a6e1
5,German Gymnasium,"[{'id': '52e81612bcbc57f1066b79f9', 'name': 'M...",1 King's Boulevard,51.532392,-0.125267,"[{'label': 'display', 'lat': 51.53239240571996...",2795,N1C 4BU,GB,London,Greater London,United Kingdom,"[1 King's Boulevard, London, Greater London, N...",NaN,Kings cross,4b792875f964a5204fed2ee3
6,Jost German Design,"[{'id': '4bf58dd8d48988d1e1931735', 'name': 'A...",13 Princes Arcade London,51.510170,-0.134421,"[{'label': 'display', 'lat': 51.51016997850715...",566,SW1Y 6DS,GB,London,NaN,United Kingdom,"[13 Princes Arcade London, London, SW1Y 6DS, U...",NaN,NaN,4f04aa02e4b04fbec49fd62d
7,Hooray’s German Hot Dogs,"[{'id': '4bf58dd8d48988d16f941735', 'name': 'H...",Covent Garden,51.511919,-0.122090,"[{'label': 'display', 'lat': 51.511919, 'lng':...",640,WC2E 8RE,GB,London,Greater London,United Kingdom,"[Covent Garden, London, Greater London, WC2E 8...",NaN,NaN,5a44f2cb2b98442bd7fbf450
8,German Doner Kebab,"[{'id': '5283c7b4e4b094cb91ec88d8', 'name': 'D...",7 Fulham Broadway,51.480196,-0.196337,"[{'label': 'display', 'lat': 51.480196, 'lng':...",5637,SW6 1AA,GB,London,Greater London,United Kingdom,"[7 Fulham Broadway, London, Greater London, SW...",NaN,NaN,583375e97ff1e41bfbb90e09
9,German Doner Kebab,"[{'id': '5283c7b4e4b094cb91ec88d8', 'name': 'D...",304 Kilburn High Rd,51.544297,-0.200867,"[{'label': 'display', 'lat': 51.544297, 'lng':...",6531,NW6 2DB,GB,London,Greater London,United Kingdom,"[304 Kilburn High Rd, London, Greater London, ...",NaN,NaN,58a8b2229168024c86918f9a


In [397]:
# filter the category for each row
#dataframe['categories'] = dataframe.apply(get_category_type, axis=1)
#dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)
# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]


In [384]:
dataframe_filtered.lng

0    -0.123265
1    -0.106242
2    -0.154989
3    -0.089977
4    -0.125259
5    -0.125267
6    -0.134421
7    -0.122090
8    -0.196337
9    -0.200867
10   -0.144363
11   -0.140136
12   -0.123915
13   -0.045689
14   -0.070470
15   -0.128703
16   -0.018924
17   -0.107878
18   -0.227240
19   -0.320906
20   -0.198384
21   -0.151877
22   -0.153374
23    0.001739
24   -0.098935
25   -0.131765
26   -0.098723
27   -0.109818
28   -0.149733
29   -0.137372
30   -0.097657
31    0.069221
32   -0.087193
33   -0.192149
34   -0.125595
35   -0.082045
36   -0.140964
37   -0.131259
38   -0.067005
39   -0.195952
40   -0.045748
41   -0.045956
42   -0.022304
43   -0.223213
44   -0.086009
45   -0.091836
46   -0.075800
47   -0.229743
48   -0.012874
49   -0.208955
Name: lng, dtype: float64

In [398]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=12)


# add a yellow circle marker to represent the center of London
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='yellow',
    popup='London',
    fill = True,
    fill_color = 'yellow',
    fill_opacity = 0.6
).add_to(venues_map)


# add German places to the map
for lat, lng, borough in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.name):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=2,
        popup=label,
        color='purple',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(venues_map)     
    
        
venues_map

In [328]:
## combining both data sets & plot them

In [399]:
map_german_london = folium.Map(location=[latitude, longitude], zoom_start=12)

# add German places to the map
for lat, lng, borough in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.name):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=2,
        popup=label,
        color='purple',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_german_london)     
    
    
for lat, lng, borough, neighborhood in zip(df_gLLL['Latitude'], df_gLLL['Longitude'], df_gLLL['Entity'], df_gLLL['Location']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_german_london)     
    
        
map_german_london

In [400]:
# end